In [3]:
# 引入rich打印
import sys
sys.path.append('../..')

from rich import print
from llm_common.set_env import set_open_ai_key,print_current_path

set_open_ai_key()
print_current_path()

7RGC8kgFXY1IT9-vxITDvXolfh3-8A

当前目录: e:\github_project\robin-learn-llm\lang_chain\c02_example_selector

In [4]:

# 设定样本

from langchain_chroma import Chroma
from chromadb import Client, config
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

print(examples)

[
    {'input': '2+2', 'output': '4'},
    {'input': '2+3', 'output': '5'},
    {'input': '2+4', 'output': '6'},
    {'input': 'What did the cow say to the moon?', 'output': 'nothing at all'},
    {
        'input': 'Write me a poem about the moon',
        'output': 'One for the moon, and one for me, who are we to talk about the moon?'
    }
]

In [5]:
# 把样本向量化，存储到向量库
to_vectorize = [" ".join(example.values()) for example in examples]

print(to_vectorize)

embeddings = OpenAIEmbeddings()

vectorstore = Chroma()
vectorstore.delete_collection()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)



[
    '2+2 4',
    '2+3 5',
    '2+4 6',
    'What did the cow say to the moon? nothing at all',
    'Write me a poem about the moon One for the moon, and one for me, who are we to talk about the moon?'
]

In [6]:
# 搜索相关示例1
search_result = vectorstore.similarity_search(query = "poem", k = 3)
print(search_result)

[
    Document(
        metadata={
            'input': 'Write me a poem about the moon',
            'output': 'One for the moon, and one for me, who are we to talk about the moon?'
        },
        page_content='Write me a poem about the moon One for the moon, and one for me, who are we to talk about the
moon?'
    ),
    Document(
        metadata={'input': 'What did the cow say to the moon?', 'output': 'nothing at all'},
        page_content='What did the cow say to the moon? nothing at all'
    ),
    Document(metadata={'input': '2+3', 'output': '5'}, page_content='2+3 5')
]

In [7]:
# 搜索相关示例2
search_result = vectorstore.similarity_search(query = "9+9", k = 2)
print(search_result)

[
    Document(metadata={'input': '2+3', 'output': '5'}, page_content='2+3 5'),
    Document(metadata={'input': '2+2', 'output': '4'}, page_content='2+2 4')
]

In [8]:
# 用示例选择器搜索向量库
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)


# The prompt template will load examples by passing the input do the `select_examples` method
selected = example_selector.select_examples({"input": "horse"})
print(f'selected: {selected}')

selected: [{'input': 'What did the cow say to the moon?', 'output': 'nothing at all'}, {'input': '2+4', 'output': 
'6'}]

In [9]:
# 用示例选择器构造提示词模板
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)


final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(final_prompt)

ChatPromptTemplate(
    input_variables=['input'],
    input_types={},
    partial_variables={},
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[],
                input_types={},
                partial_variables={},
                template='You are a wondrous wizard of math.'
            ),
            additional_kwargs={}
        ),
        FewShotChatMessagePromptTemplate(
            example_selector=SemanticSimilarityExampleSelector(
                vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001A7D60CA090>,
                k=2,
                example_keys=None,
                input_keys=None,
                vectorstore_kwargs=None
            ),
            input_variables=['input'],
            input_types={},
            partial_variables={},
            example_prompt=ChatPromptTemplate(
                input_variables=['input', 'output'],
                input_types={},
                partial_variables={},
                messages=[
                    HumanMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['input'],
                            input_types={},
                            partial_variables={},
                            template='{input}'
                        ),
                        additional_kwargs={}
                    ),
                    AIMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['output'],
                            input_types={},
                            partial_variables={},
                            template='{output}'
                        ),
                        additional_kwargs={}
                    )
                ]
            )
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['input'],
                input_types={},
                partial_variables={},
                template='{input}'
            ),
            additional_kwargs={}
        )
    ]
)

In [10]:
# 使用提示词模板构造提示词
print(final_prompt.invoke(input="What's 3+3?"))

ChatPromptValue(
    messages=[
        SystemMessage(content='You are a wondrous wizard of math.', additional_kwargs={}, response_metadata={}),
        HumanMessage(content='2+3', additional_kwargs={}, response_metadata={}),
        AIMessage(content='5', additional_kwargs={}, response_metadata={}),
        HumanMessage(content='2+2', additional_kwargs={}, response_metadata={}),
        AIMessage(content='4', additional_kwargs={}, response_metadata={}),
        HumanMessage(content="What's 3+3?", additional_kwargs={}, response_metadata={})
    ]
)